
1. CSR matrix가 정상적으로 만들어졌다.	

- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.	

- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.	

- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.

필요 라이브러리 버전 확인

In [365]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


1) 데이터 준비와 전처리

In [366]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [367]:
# 지저분하니 timestamp를 지웁니다.
ratings = ratings.drop('timestamp', axis=1)

- 별점을 시청횟수로 해석
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

In [368]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [369]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [370]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [371]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [372]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수   




In [373]:
num_movie = ratings.movie_id.nunique()
print(num_movie)

3628


- ratings에 있는 유니크한 사용자 수   


In [374]:
num_user = ratings.user_id.nunique()
print(num_user)

6039


- 가장 인기 있는 영화 30개(인기순)


In [375]:
sum_counts = ratings[['movie_id', 'counts']].groupby('movie_id').sum()
sum_counts = sum_counts.sort_values(by='counts',ascending=False)

pd.merge(sum_counts.head(30), movies, how='left', left_on='movie_id', right_on='movie_id')

,movie_id,counts,title,genre
0,2858,14449,american beauty (1999),Comedy|Drama
1,260,13178,star wars: episode iv - a new hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,12648,star wars: episode v - the empire strikes back...,Action|Adventure|Drama|Sci-Fi|War
3,2028,11348,saving private ryan (1998),Action|Drama|War
4,1210,11303,star wars: episode vi - return of the jedi (1983),Action|Adventure|Romance|Sci-Fi|War
5,1198,11179,raiders of the lost ark (1981),Action|Adventure
6,593,11096,"silence of the lambs, the (1991)",Drama|Thriller
7,2571,10903,"matrix, the (1999)",Action|Sci-Fi|Thriller
8,2762,10703,"sixth sense, the (1999)",Thriller
9,589,10513,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller


3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [376]:
movies[movies['title'].isin(my_movie_name)]

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1250,1270,back to the future (1985),Comedy|Sci-Fi
1539,1580,men in black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"matrix, the (1999)",Action|Sci-Fi|Thriller
2647,2716,ghostbusters (1984),Comedy|Horror


In [357]:
ratings.user_id.max()

6041

In [377]:
my_movie_name = ['men in black (1997)', 'toy story (1995)', 'back to the future (1985)', 'matrix, the (1999)', 'ghostbusters (1984)']
favorite_movie_id = movies[movies['title'].isin(my_movie_name)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'counts':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1,5
1250,6041,1270,5
1539,6041,1580,5
2502,6041,2571,5
2647,6041,2716,5


4) CSR matrix를 직접 만들어 봅시다.

In [380]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836479,5854,3026,4,slaughterhouse (1987),Horror
836480,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836481,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary


In [381]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [382]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [383]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [384]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [385]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [390]:
my_vector, toy_vector = als_model.user_factors[6041], als_model.item_factors[1]

In [391]:
my_vector

array([-2.40830913e-01,  1.17977135e-01, -2.62031853e-01,  2.14442924e-01,
       -3.79845053e-01,  3.89418989e-01,  1.99863121e-01, -3.44462216e-01,
        7.46790051e-01,  2.47509032e-01, -2.49493361e-01, -7.26415932e-01,
       -4.28468585e-01,  7.72660971e-02, -2.69844323e-01,  5.08461833e-01,
       -2.06219688e-01, -2.51630664e-01, -1.35404766e+00, -1.68539315e-01,
        6.95803523e-01, -6.58279598e-01, -4.06073213e-01,  4.62980986e-01,
        6.65689349e-01,  1.88832343e-01, -6.84042513e-01, -3.49217921e-01,
       -2.36568406e-01,  3.51400912e-01,  8.51395428e-02,  3.45490426e-01,
       -8.87314558e-01, -3.45763445e-01,  4.29575264e-01,  3.11466873e-01,
       -2.88337767e-01, -2.72963405e-01, -2.45172709e-01,  5.28105676e-01,
        8.75912786e-01, -3.37298989e-01,  1.02460332e-01,  2.39694715e-01,
        1.17811710e-01, -2.86779821e-01, -1.01979496e-02,  8.03685665e-01,
       -2.13854522e-01, -4.47626077e-02,  5.16899109e-01,  3.79152596e-01,
        1.04816295e-01,  

In [392]:
toy_vector

array([ 0.01590458, -0.00436746, -0.02066138,  0.00038828,  0.00796153,
        0.01136898,  0.00779137, -0.00897455, -0.00289083, -0.01236635,
        0.00861281, -0.01912149,  0.02443153,  0.02309897,  0.02313603,
       -0.00498823, -0.00830782, -0.00293121, -0.01256493, -0.01774013,
        0.01012053, -0.01439927,  0.01147913,  0.02390416,  0.03052603,
        0.00440473,  0.00569844, -0.00691301,  0.01276047, -0.00642949,
        0.0323894 ,  0.01428453, -0.02435508,  0.00610903,  0.00299524,
        0.03241065, -0.01403356, -0.00967323, -0.00438214,  0.04552586,
        0.0214505 , -0.01083174,  0.00343096,  0.02137277, -0.01315007,
       -0.01594269, -0.02581326,  0.03561097,  0.01672006,  0.0277352 ,
        0.03542964,  0.00722722,  0.00253053, -0.0099211 ,  0.01242378,
        0.02485139, -0.01276578,  0.02155077,  0.03178661,  0.00139788,
       -0.01564148, -0.0211383 , -0.01794778, -0.01660759, -0.02004435,
        0.02538272,  0.00534516, -0.01195558,  0.00307352, -0.01

In [395]:
np.dot(my_vector, toy_vector)

0.5239203

- 스타워즈와 선호도를 비교해보겠습니다

In [396]:
star_vector = als_model.item_factors[260]
np.dot(my_vector, star_vector)

0.25299394

7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [400]:
favorite_movie = 'shakespeare in love (1998)'
movie_id = movies[movies['title']==favorite_movie]['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(2396, 1.0000001),
 (1265, 0.53916943),
 (2599, 0.48966327),
 (2336, 0.48108295),
 (2997, 0.43901366),
 (2858, 0.43114924),
 (357, 0.40987578),
 (1641, 0.38704547),
 (1579, 0.38526163),
 (2355, 0.3711587),
 (1, 0.36275458),
 (2762, 0.36235696),
 (39, 0.3561087),
 (3114, 0.3556735),
 (2686, 0.34668744)]

In [401]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
105,107,muppet treasure island (1996),Adventure|Children's|Comedy|Musical
493,497,much ado about nothing (1993),Comedy|Romance
630,635,"family thing, a (1996)",Comedy|Drama
665,671,mystery science theater 3000: the movie (1996),Comedy|Sci-Fi
900,912,casablanca (1942),Drama|Romance|War
974,986,fly away home (1996),Adventure|Children's
1063,1079,"fish called wanda, a (1988)",Comedy
1064,1080,monty python's life of brian (1979),Comedy
1120,1136,monty python and the holy grail (1974),Comedy
1176,1193,one flew over the cuckoo's nest (1975),Drama


8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [402]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.49493125),
 (480, 0.48293322),
 (3114, 0.4270981),
 (3175, 0.32915193),
 (1097, 0.32913595),
 (1265, 0.3081707),
 (2916, 0.30772108),
 (1197, 0.30745453),
 (2762, 0.2974563),
 (2791, 0.2891612),
 (2355, 0.2861771),
 (356, 0.2810215),
 (1210, 0.26809844),
 (457, 0.26058316),
 (1240, 0.25328532),
 (1196, 0.2531298),
 (260, 0.25299394),
 (2918, 0.25292152),
 (2987, 0.24465054),
 (2797, 0.24441877)]

In [404]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
257,260,star wars: episode iv - a new hope (1977),Action|Adventure|Fantasy|Sci-Fi
352,356,forrest gump (1994),Comedy|Romance|War
453,457,"fugitive, the (1993)",Action|Thriller
476,480,jurassic park (1993),Action|Adventure|Sci-Fi
585,589,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
1081,1097,e.t. the extra-terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,star wars: episode v - the empire strikes back...,Action|Adventure|Drama|Sci-Fi|War
1179,1197,"princess bride, the (1987)",Action|Adventure|Comedy|Romance
1192,1210,star wars: episode vi - return of the jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1220,1240,"terminator, the (1984)",Action|Sci-Fi|Thriller


MF model기반 movie 추천 시스템을 만들었다.

- 내가 좋아하는 토이스토리 영화에 대한 선호도는 약 0.52, 보지 않은 starwars의 선호도는 약 0.25로 도출되었다.

- 유사영화 추천 결과 좋아한다고 입력한 'shakespeare in love (1998)'의 장르와 비슷한 comedy, romance장르가 다수 추천되었다.